# Python-Code paketieren

(vgl. [Packaging Python Projects](https://packaging.python.org/tutorials/packaging-projects/))

Wenn wir eine Python-Anwendung mit mehreren Modulen oder auch nur ein einzelnes Skript mit Abhängigkeiten erstellt haben, möchten wir diese in ein Paket zusammenbinden, damit wir diese Funktionalität leicht auf anderen Systemen installieren können.

Beispiel: [surveyresponse.py](surveyresponse.py)

Dieses Paket benötigt unter Linux keine Abhängigkeiten, auf Windows jedoch `pywin32`.

Ein Python-Paket hat üblicherweise folgende Struktur (Beispiel Survey Response-Applikation):
```
survey_response/
├── README.md
├── pyproject.toml
├── survey_response.py
└── test/
    └── test_survey_response.py
```
Im obersten Verzeichnis liegt die Wurzel des (Git-) Repositories. Es enthält ein Unterverzeichnis mit dem gleichen Namen sowie das File *pyproject.toml*.
In *README.md* sollte Zweck und eine kurze Anleitung angegeben werden.

Das gleichnamige Unterverzeichnis ist das eigentliche Python-Modul. In diesem Beispiel liegt der Code im File *survey_response.py* (single file Modul).

Das File *pyproject.toml* enthält Projekt-Metadaten, welche zur Paketierung benötigt werden (ausführlichere Anleitung auf der Seite von [setuptools](https://setuptools.pypa.io/en/latest/userguide/declarative_config.html)).

**pyproject.toml** ([doc](https://pip.pypa.io/en/stable/reference/build-system/pyproject-toml/))
* deklariert welches build system verwendet wird (hier: setuptools)
* deklariert abhängigkeiten, Projekt-Metainformation, Entry-Points
* enthält konfiguration von code quality tools wie pylint, tox, formatting

```toml
[build-system]
requires = ["setuptools >= 40.6.0", "wheel"]
build-backend = "setuptools.build_meta"
```

**pyproject.toml** kann nicht nur mit [setuptools](https://setuptools.pypa.io) eingesetzt werden, sondern ist universell für jede Paketierungslösung gültig.
Die prominenteste Alternative als Paketierungs-Tool ist [Poetry](https://python-poetry.org/).

```toml
[build-system]
requires = ["setuptools >= 40.6.0", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "survey_response"
version = 0.0.1
authors = [
    {name = "Chuck Norris", email = "python.sysadmins@id.ethz.ch"},
]
description = "Tool to collect mail survey responses in a folder in a csv file."
readme = {file = "README.md", content_type = "text/markdown"}
license = {text = "BSD-3-Clause"}
classifiers = [
    "Programming Language :: Python :: 3",
]
dependencies = [
    "importlib-metadata; platform_system=='Windows'",
]

[project.scripts]
mail_dir2csv = "survey_response:main"

[options]
zip_safe = False
packages = find:
```

- **name**: Der *distribution name* der des Pakets. Er muss eindeutig sein. Dies kann z.B. erreicht werden, wenn ein beliebiger Name mit dem Usernamen ergänzt wird.
- **version**: Version des Pakets.
- **description**: Kurze, einzeilige Zusammenfassung des Pakets.
- **readme**: Kann via `file=` den Inhalt von `README.md` einlesen und als ausführliche Version von **description** verwenden.
- **dependencies**: Liste aller importierten Python-Pakete, welche im Paket aufgenommen werden sollen. Diese Liste kann entweder manuell oder mit `find:` automatisiert erzeugt werden.
- **dependencies**: Abhängigkeiten zu externen Python packages. Hier wird mit `platform_system=='Windows'` angegeben, dass die Abhängigkeit nur unter Windows benötigt wird.

Ist die Paketstruktur mit Code und Setup-Konfiguration bereit, kann das Paket erstellt werden. Zu diesem Zweck muss im Wurzelverzeichnis des Pakets folgender Befehl aufgerufen werden.:

```sh
pip install .
```

Damit ist das Python-Paket lokal installiert. Dies kann mit `pip list` überprüft werden.

Mit folgenden Befehlen kann ein Archiv des Python-Pakets erzeugt werden:

```sh
pip install --upgrade build
python -m build
```
Mit dem ersten Befehl wird sichergestellt, dass die aktuellsten Versionen von *build* installiert ist. Mit dem zweiten Befehl wird das Paket als *tar*-File und als *wheel* verpackt und im Unterverzeichnis *dist* abgelegt.

Dieses File kann nun beispielsweise auf den Python Paket-Index (*PyPI*) [https://pypi.org/](https://pypi.org/) geladen werden. Dieser Upload auf *PyPI* erfolgt am besten mit Hilfe von [twine](https://twine.readthedocs.io/en/latest/).

## Versionsnummern

Die Versionsnummer eines Pakets, welches auf *PyPI* geladen werden soll, muss sich von einem existierenden Paket (mit dem gleichen Namen) unterscheiden. Sinnvollerweies wird die Versionsnummer im Format gemäss
[*Semantic Versioning*](https://semver.org/) angegeben: MAJOR.MINOR.PATCH.

Vor einem wichtigen Release (z.B. *1.0.0*) müssen unter Umständen sog. Alpha- und Beta-Release veröffentlicht werden. Eine gültie Versionshistorie kann wie folgt aussehen: `0.9.99` -> `1.0.0-alpha.1` -> `1.0.0-alpha.2` -> `1.0.0-beta.1` -> `1.0.0`.

## Kommandozeilen-Tool paketieren

Wenn man `ethz-iam-webservice` installiert:
```sh
pip install ethz-iam-webservice
```
ist danach automatisch ein Kommandozeilen-Tool `iam` verfügbar.
Wie macht man das?

Das kann mit dem *project.scripts* in *pyproject.toml* erreicht werden:
```toml
[project.scripts]
iam = ethz_iam_webservice.main:cli
```

`setuptools` erstellt automatisch ein kleines Python Skript `iam` (wrapper script) im `bin/`
Unterordner vom Home Ordner (`~/.local/bin` (🐧/🍏) / `AppData\Roaming\bin` (🪟)) oder vom virtual environment falls im Einsatz. 
Das Skript führt `ethz_iam_webservice.main:cli` aus:

```
ethz_iam_webservice.main:cli
      ^              ^    ^
      |              |    |
  module name   main.py  method name
```

Im Beispiel von `survey_response` was nur aus einem py File besteht, verwenden wir:

```toml
[project.scripts]
mail_dir2csv = "survey_response:main"
```

Weiterführende Informationen auf [setuptools.pypa.io](https://setuptools.pypa.io/en/latest/userguide/entry_point.html).

## Multi file Modules

Im obigen Beispiel haben wir nur ein einzelnes Skript [surveyresponse.py](surveyresponse.py) paketiert.
Wenn das Prokekt wächst und das Skript in mehrere Files gegliedert wird, verändert sich die Ordnerstruktur wie folgt:
```
survey_response/
├── README.md
├── pyproject.toml
├── survey_response
|   ├── __init__.py
|   ├── __main__.py
|   ├── file_module_a.py
|   ├── folder_module_b
|   |   ├── __init__.py
|   |   ├── file_sub_module.py
|   |   └── ...
|   └── ...
|
└── test/
    └── test_survey_response.py
```

`project.scripts` muss lediglich zu
```toml
[project.scripts]
mail_dir2csv = "survey_response.__main__:main"
```
verändert werden.

## Example (standalone script)

```
examples/stand_alone_script/
├── pyproject.toml
└── user.py
```
`user.py`
```python
def main():
    print("🎉")
```

`pyproject.toml`
```toml
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "user"
version = "0.0.1"

[project.scripts]
user-ctl = "user:main"
```

## Example (multi file package)

```
examples/multi_file_package/
├── pyproject.toml  # <- 1 line changed
└── user/
    ├── __init__.py # <- empty
    ├── user.py
    ├── gui.py
    └── cli.py

```
`user.py`
```python
def add_user():
    print("➕👤")
```
`cli.py`
```python
from user.user import add_user
def main():
    print("CLI")
    add_user()
```
`gui.py`
```python
from user.user import add_user
def main():
    print("GUI")
    add_user()
```

`pyproject.toml`
```diff
 version = "0.0.1"
 
 [project.scripts]
-user-ctl = "user:main"
+user-ctl = "user.cli:main"
```

### Exercise: install your command-line tool locally

- [ ] use `pip install . --prefix local_python_packages` to specify a folder where packages are installed
   - Notice how a `local_python_packages` folder is created
- [ ] add the `local_python_packages` folder to your `PYTHONPATH` (make sure you pick the right Python version!)
  ```sh
  export PYTHONPATH="$PWD/$(find local_python_packages -name site-packages)"
  ```
- [ ] add this path to your `PATH`:
  ```sh
  export PATH="$PWD/local_python_packages/bin:$PATH"
  ```
- [ ] try out the new utility:
  ```sh
  user-ctl
  ```